# Transfromacion de Orders data convertirlo a un  objeto json

In [0]:
df_orders=spark.table("gizmobox.bronze.py_orders")

In [0]:
display(df_orders)

In [0]:
from pyspark.sql import functions as F
df_1=(
df_orders.select(
    F.regexp_replace("value",'"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "$1"').alias('fixed_value'),
   
)

)
display(df_1)

In [0]:

df_schema=(
    df_1.select(
        F.schema_of_json(F.col('fixed_value')).alias('schema')
    )
)

display(df_schema.limit(1))

In [0]:
schema_orders='''STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'''

In [0]:
df_json_orders=(
    df_1.select(
        F.from_json(F.col('fixed_value'),schema_orders).alias('json_value')
    )
)

In [0]:
display(df_json_orders.limit(5))

In [0]:
df_json_orders.writeTo("gizmobox.silver.py_orders_json").createOrReplace()